In [28]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [16]:
!pip install mlxtend --quiet
!pip install pandas numpy matplotlib --quiet

In [17]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [18]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/Groceries_dataset.csv')
print("Dữ liệu ban đầu: ", data.shape)
print("5 Bộ dữ liệu đầu tiên", data.head())

Dữ liệu ban đầu:  (38765, 3)
5 Bộ dữ liệu đầu tiên    Member_number        Date   itemDescription
0           1808  21-07-2015    tropical fruit
1           2552  05-01-2015        whole milk
2           2300  19-09-2015         pip fruit
3           1187  12-12-2015  other vegetables
4           3037  01-02-2015        whole milk


Làm sạch dữ liệu cơ bản

In [19]:
import re
data['Date'] = pd.to_datetime(data['Date'], dayfirst = True, errors = 'coerce')
data = data.dropna(subset = ['Date'])
data['itemDescription'] = data['itemDescription'].astype(str)
data = data[data['itemDescription'].str.strip() != '']
def clean_item(s):
  s = str(s).lower().strip()
  s = re.sub(r'[/\\]', ' ', s)   # thay / hoặc \ bằng khoảng trắng
  s = re.sub(r'[^a-z0-9\s\-\&]', ' ', s) # chỉ giữ lại chữ, số, -, &, khoảng trắng
  s = re.sub(r'\s*-\s*', '-', s) # chuẩn hoá dấu gạch ngang
  s = re.sub(r'\s*&\s*', '&', s)# chuẩn hoá dấu &
  s = re.sub(r'\s+', ' ', s).strip() # loại bỏ khoảng trắng thừa
  return s
data['item_clean'] = data['itemDescription'].apply(clean_item)

Xây dựng mã giao dịch


In [20]:
data['Member_number'] = data['Member_number'].astype(str)
data['TransID'] = data['Member_number'] + '_' + data['Date'].dt.strftime('%Y-%m-%d')

Gom các mặt hàng trong cùng giao dịch

In [21]:
transactions = (
    data.groupby('TransID')['item_clean'].apply(lambda x: list(dict.fromkeys(x))).tolist()
)

Đếm và lọc mặt hàng hiếm

In [22]:
from collections import Counter
cnt = Counter(it for t in transactions for it in t)
print("Số giao dịch ban đầu: ", len(transactions))
print("Top items: ", cnt.most_common(15))
min_count = 5
freq_items = {it for it, c in cnt.items() if c >= min_count}
transactions = [[it for it in t if it in freq_items] for t in transactions]
transactions = [t for t in transactions if len(t) >= 1]

Số giao dịch ban đầu:  14963
Top items:  [('whole milk', 2363), ('other vegetables', 1827), ('rolls buns', 1646), ('soda', 1453), ('yogurt', 1285), ('root vegetables', 1041), ('tropical fruit', 1014), ('bottled water', 908), ('sausage', 903), ('citrus fruit', 795), ('pastry', 774), ('pip fruit', 734), ('shopping bags', 712), ('canned beer', 702), ('bottled beer', 678)]


Chuyển sang dạng one-hot encoding chuẩn bị cho khai phá luật kết hợp

In [23]:
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_onehot = pd.DataFrame(te_ary, columns=te.columns_)
df_onehot.to_csv('groceries_onehot.csv', index=False)
print("One-hot saved, shape: ", df_onehot.shape)

One-hot saved, shape:  (14963, 163)


Chạy apriori

In [29]:
from mlxtend.frequent_patterns import apriori, association_rules
min_support = 0.01
frequent_itemsets = apriori(
    df_onehot,
    min_support = min_support,
    use_colnames=True, #Giữ tên sản phẩm thay vì index cột
    max_len = 3 # Chỉ xét tối đa 3 sản phẩm / tập
)
# Sắp xếp để xem các tập phổ biến nhất
frequent_itemsets = frequent_itemsets.sort_values(by = "support", ascending=False)
print("Số tập phổ biến tìm được: ", frequent_itemsets.shape[0])
print("Top 10 tâp phổ biến nhất: ")
print(frequent_itemsets.head(10))
#Lưu lai
frequent_itemsets.to_csv("frequent_itemsets.csv", index = False)
print("Đã lưu frequent_itemsets.csv")

Số tập phổ biến tìm được:  69
Top 10 tâp phổ biến nhất: 
     support            itemsets
62  0.157923        (whole milk)
39  0.122101  (other vegetables)
45  0.110005        (rolls buns)
51  0.097106              (soda)
63  0.085879            (yogurt)
46  0.069572   (root vegetables)
56  0.067767    (tropical fruit)
4   0.060683     (bottled water)
48  0.060349           (sausage)
14  0.053131      (citrus fruit)
Đã lưu frequent_itemsets.csv


Sinh luật kết hợp từ các tập

In [35]:
#Tạo các luật dựa trêm chỉ số "confidence"
#min_threshold ở đây chính là min_confidence
rules = association_rules(
    frequent_itemsets,
    metric = "confidence",
    min_threshold = 0.1 #Chỉ lấy cấc luật có độ tin cậy >= 50%
)
# Thêm độ dài vế trái để tiện lọc luật dạng "mua combo -> mua thêm gì"
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
#Sắp xếp luật theo lift (độ "thú vị", >1 nghĩa là có liên kết dương)
rules_sorted = rules.sort_values(by = "lift", ascending = False)
print("Tổng số luật tạo ra: ", rules_sorted.shape[0])
print("Top 10 luật lift cao nhất")
cols_show=[
    "antecedents", "consequents",
    "support", "confidence","lift",
    "antecedent_len"
]
print(rules_sorted[cols_show].head(10))
#lưu file
rules_sorted.to_csv("all_association_rules.csv", index = False)
print("Đã lưu all_association_rules.csv")

Tổng số luật tạo ra:  4
Top 10 luật lift cao nhất
          antecedents   consequents   support  confidence      lift  \
3            (yogurt)  (whole milk)  0.011161    0.129961  0.822940   
1        (rolls buns)  (whole milk)  0.013968    0.126974  0.804028   
0  (other vegetables)  (whole milk)  0.014837    0.121511  0.769430   
2              (soda)  (whole milk)  0.011629    0.119752  0.758296   

   antecedent_len  
3               1  
1               1  
0               1  
2               1  
Đã lưu all_association_rules.csv


Lọc ra những luật mạnh

In [31]:
#lift > 1.2 : mối quan hệ mua kèm thực sự mạnh
#confidence >= 0.3 : tin cậy ( ít nhất 30% người mua A cũng mua B)
#support >= 0.01 : không quá hiếm, có ý nghĩa thực tế
strong_rules = rules_sorted[
    (rules_sorted["lift"] > 1.2) &
    (rules_sorted["confidence"] >= 0.3) &
    (rules_sorted["support"] >= 0.01)
][cols_show]
print("Các luật mạnh sau khi lọc: ")
print(strong_rules.head(20))

#lưu
strong_rules.to_csv("strong_rules.csv", index = False)
print("Đã lưu strong_rules")

Các luật mạnh sau khi lọc: 
Empty DataFrame
Columns: [antecedents, consequents, support, confidence, lift, antecedent_len]
Index: []
Đã lưu strong_rules
